In [15]:
import warnings
warnings.filterwarnings('ignore')
#%matplotlib inline
import numpy as np
import pandas as pd

In [16]:
##################  Загрузка данных
#Data1 = pd.read_csv('D:\Coding\data\phone_user_review_file_1.csv', encoding= 'ANSI')
#Data2 = pd.read_csv('D:\Coding\data\phone_user_review_file_2.csv', encoding= 'ANSI')
#Data3 = pd.read_csv('D:\Coding\data\phone_user_review_file_3.csv', encoding= 'ANSI')
#Data4 = pd.read_csv('D:\Coding\data\phone_user_review_file_4.csv', encoding= 'ANSI')
#Data5 = pd.read_csv('D:\Coding\data\phone_user_review_file_5.csv', encoding= 'ANSI')
Data6 = pd.read_csv('D:\Coding\data\phone_user_review_file_6.csv', encoding= 'ANSI')

#total_data = pd.concat([Data1,Data2, Data3, Data4, Data5, Data6], axis=0)
total_data = Data6

total_data.shape

(163837, 11)

In [17]:
data = total_data[(total_data.lang == 'en')]
data = data[['score', 'extract']]
data = data.dropna()

data.shape

(49093, 2)

In [1]:
import tensorflow_hub as hub
import tensorflow as tf
import bert
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math

In [2]:
max_seq_length = 128  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")

input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")

segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")

bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)

pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

In [3]:
model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

In [4]:
# See BERT paper: https://arxiv.org/pdf/1810.04805.pdf
# And BERT implementation convert_single_example() at https://github.com/google-research/bert/blob/master/run_classifier.py

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [5]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [6]:
s = "This is a nice sentence."


In [83]:
def review_to_wordlist(review):
    stokens = tokenizer.tokenize(review)
    stokens = ["[CLS]"] + stokens + ["[SEP]"]
    
    return (stokens)

In [84]:
data['content_clear'] = data['extract'].apply(review_to_wordlist)

101

In [21]:
#stokens = tokenizer.tokenize(data['extract'][0])

In [42]:
#stokens = ["[CLS]"] + stokens + ["[SEP]"]

In [107]:
def inputs(review):
    input_ids = get_ids(review, tokenizer, max_seq_length)
    #input_masks = get_masks(review, max_seq_length)
    #input_segments = get_segments(review, max_seq_length)
    
    return (input_ids)

In [108]:
data['content_clear_2'] = data['content_clear'].apply(inputs)

In [76]:
data['content_clear_2']

0         [101, 1045, 1005, 2310, 2018, 1996, 3042, 2005...
1         [101, 2000, 2022, 3154, 2009, 2003, 2025, 1996...
2         [101, 2092, 1045, 2293, 2023, 3042, 1012, 1045...
3         [101, 1045, 2031, 2018, 2026, 12753, 2005, 219...
4         [101, 1045, 2031, 2018, 2023, 12753, 3042, 200...
                                ...                        
163814    [101, 2023, 2003, 2019, 10303, 3042, 1012, 200...
163815    [101, 2123, 2102, 8572, 2130, 6195, 2000, 4965...
163816    [101, 1996, 3042, 1999, 1996, 3861, 1006, 4983...
163817    [101, 2383, 3728, 4149, 1037, 4098, 2239, 2953...
163818    [101, 1045, 3728, 4149, 1037, 29536, 2850, 148...
Name: content_clear, Length: 49093, dtype: object

In [105]:
#input_ids = get_ids(data['content_clear'][3], tokenizer, max_seq_length)
input_masks = get_masks(data['content_clear'][0], max_seq_length)
input_segments = get_segments(data['content_clear'][0], max_seq_length)

In [109]:
print(stokens)
print(input_ids)
print(input_masks)
print(input_segments)

['[CLS]', 'i', "'", 've', 'had', 'the', 'phone', 'for', 'awhile', 'and', 'it', "'", 's', 'a', 'pretty', 'good', 'phone', '.', 'the', 'call', 'quality', ',', 'response', 'to', 'sms', 'and', 'mm', '##s', 'much', 'faster', ',', 'and', 'data', '/', '3', '##g', 'runs', 'like', 'it', 'is', 'on', 'a', '4', '##g', 'network', '.', 'this', 'phone', 'does', 'pretty', 'much', 'like', 'an', 'android', 'except', 'without', 'the', 'wi', '##dgets', 'and', 'market', '.', '[SEP]']
[101, 1045, 2031, 2018, 2026, 12753, 2005, 2195, 2706, 2085, 1998, 1045, 2031, 2000, 6449, 2009, 2003, 1037, 3835, 3042, 1010, 2021, 2003, 2428, 11158, 1999, 2752, 1012, 1996, 4773, 16602, 2003, 3835, 1998, 4248, 1012, 1996, 3291, 2007, 2009, 2003, 1996, 2755, 2008, 2028, 2064, 2025, 7276, 1998, 3828, 1037, 3861, 2030, 3793, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [110]:
import numpy as np

input_ids2 = np.array(input_ids)
input_masks2 = np.array(input_masks)
input_segments2 = np.array(input_segments)

In [111]:
pool_embs, all_embs = model.predict([[input_ids2],[input_masks2],[input_segments2]])

In [112]:
def square_rooted(x):
    return math.sqrt(sum([a*a for a in x]))


def cosine_similarity(x,y):
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return numerator/float(denominator)

In [113]:
cosine_similarity(pool_embs[0], all_embs[0][0])

0.04704362890868893